In [77]:
%load_ext autoreload
%autoreload 2


%matplotlib inline
%cd C:\MAD4AG

from tqdm.auto import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [78]:
filename = r'.\dbs\twins\multiple_days_03-21.pkl'

combined_df= pd.read_pickle(filename)
#combined_df.columns


In [79]:
df_clusters=pd.read_parquet(f'./dbs/intermediate/df_selected_clusters_activities.parquet')

In [80]:
df_survey_act = pd.read_pickle(f'./dbs/intermediate/survey_act_weekday_modified.pkl')

## generate activity schedule for simulation #1
- by merging survey activity sequence table
- the survey activity sequence table brings activity sequences and activity start end times

In [81]:
day = '4'

In [82]:
df_act_schedule_0 = pd.merge(combined_df[['uid', 'employee', 'Deso', 'wt', 'wt_p', 'sub_id_'+day,'act_seq_simp_'+day]], df_survey_act[['sub_id', 'purpose','h_s','h_e']], left_on='sub_id_'+day,right_on='sub_id')

df_act_schedule_0['act_seq'] = df_act_schedule_0.groupby('uid').cumcount()



### add sequence information of other activities

In [83]:
df_act_schedule_0['other_order']= 0

df_act_schedule_0['other_order'][df_act_schedule_0.purpose=='Other'] = df_act_schedule_0[df_act_schedule_0.purpose=='Other'].groupby('uid').cumcount()

df_act_schedule_0['other_order']=df_act_schedule_0['other_order'].astype(str)

df_act_schedule_0['purpose'][df_act_schedule_0.purpose=='Other']= df_act_schedule_0['purpose'][df_act_schedule_0.purpose=='Other'] + df_act_schedule_0['other_order'][df_act_schedule_0.purpose=='Other']

df_act_schedule_0.drop(columns=['other_order'] ,inplace=True)


## re-organise df_cluster table by the other activities order and repetition

In [84]:
df_clusters_0 = df_clusters[['uid','cluster','other_clusters_'+day,'selection_'+day, 'X','Y','cluster_lat','cluster_lng']][df_clusters['selection_'+day] != 'non']

In [85]:
df_clusters_0_other = df_clusters_0[df_clusters_0['selection_'+day] =='Other']


df_other_org = df_clusters_0_other[['uid', 'other_clusters_'+day]]
df_other_org= df_other_org.drop_duplicates(subset='uid')


In [86]:
# it will transform each element of the list in the cloumn 'other_cluster_O' to a row.

df_other_org = df_other_org.explode('other_clusters_'+day)

df_other_org.rename(columns={'other_clusters_'+day:'cluster'}, inplace=True)

df_other_org['other_order'] = df_other_org.groupby('uid').cumcount()

In [87]:
df_other_org = pd.merge(df_other_org, df_clusters_0_other, on=['uid', 'cluster'],how='left')

# add sequence information of other activities

df_other_org['other_order'] = df_other_org['other_order'].astype(str)

df_other_org['selection_'+day]= df_other_org['selection_'+day]+ df_other_org['other_order']

df_other_org.drop(columns=['other_order'] ,inplace=True)

In [88]:
df_clusters_0 = df_clusters_0[df_clusters_0['selection_'+day] !='Other']

In [89]:
df_clusters_0 = pd.concat([df_clusters_0, df_other_org], ignore_index=True)

In [90]:
df_clusters_0.drop(columns=['other_clusters_'+day] ,inplace=True)

In [91]:
df_act_schedule_0 = pd.merge(df_act_schedule_0, df_clusters_0, left_on=['uid','purpose'],right_on=['uid','selection_'+day], how='left')

In [92]:
df_act_schedule_0.drop(columns=['selection_'+day] ,inplace=True)


In [93]:
df_act_schedule_0.rename(columns={'cluster_lat':'lat','cluster_lng':'lng'}, inplace=True)

In [94]:
df_act_schedule_0['purpose'][ ~df_act_schedule_0['purpose'].isin(['Home', 'Work'])] = df_act_schedule_0['purpose'][ ~df_act_schedule_0['purpose'].isin(['Home', 'Work'])].str[:5]

In [95]:
file = r'.\dbs\twins\%s-schedule-%s.pkl'%(filename[26:31],day)

df_act_schedule_0.to_pickle(file)

# code ends here

In [298]:
df_clusters_distances = df_clusters[['uid','cluster','other_clusters_'+day,'selection_'+day, 'X','Y','cluster_lat','cluster_lng','distance']][df_clusters['selection_'+day] != 'non']

In [299]:
from scipy.spatial import distance
from tqdm.auto import tqdm



def other_dist(data):
    #home_dist = distance.cdist(data[['X', 'Y']][data.purpose == 'Home'].head(1), data[['X', 'Y']],  metric='euclidean')

    if len(data[data['selection_'+day] == 'Work'])>0:
        work_dist = distance.cdist(data[['X', 'Y']][data['selection_'+day] == 'Work'].head(1), data[['X', 'Y']],  metric='euclidean')
        #data['home_dist']= home_dist[0]
        data['work_dist']= work_dist[0]




    return data


tqdm.pandas()
df_clusters_distances = df_clusters_distances.groupby('uid').progress_apply(other_dist)

  0%|          | 0/258667 [00:00<?, ?it/s]

In [303]:
df_clusters_distances['work_dist']= df_clusters_distances['work_dist'].fillna(0)

In [306]:
ppl_other = df_act_schedule_0[df_act_schedule_0.purpose=='Other'].groupby(['uid']).size().reset_index(name='count')
ppl_other = ppl_other['uid'][ppl_other['count']>1].unique()

In [307]:

ppl_other =df_act_schedule_0[df_act_schedule_0.uid.isin(ppl_other)]

ppl_other= ppl_other.sort_values(by=['uid','act_seq'])

In [308]:
ppl_other = pd.merge(ppl_other, df_clusters_distances[['uid','cluster','distance','work_dist']], on= ['uid','cluster'], how='left')

In [311]:
ppl_other.columns

Index(['uid', 'employee', 'Deso', 'wt', 'wt_p', 'sub_id_4', 'act_seq_simp_4',
       'sub_id', 'purpose', 'h_s', 'h_e', 'act_seq', 'cluster', 'X', 'Y',
       'lat', 'lng', 'distance', 'work_dist'],
      dtype='object')

In [313]:
ppl_other_new = ppl_other[['uid','purpose', 'act_seq', 'cluster', 'distance', 'work_dist']]

In [335]:
def seq_modifier(data):
    other_candidates = data[data['purpose']=='Other']
    for i in range(0, len(data) - 1, 1):
        #collector = []
        if data.iloc[i, 1] in ['Home']:
            collector = []
            #collector = collector.append(i)
        if data.iloc[i, 1] in ['Other']:
            collector += i
            print(data.iloc[collector, 1])

                #print(collector)

            # if data.iloc[i+1, 1] in ['Other']:
            #
            #     home_candidate = other_candidates.sort_values(by=['distance']).head(1)
            #     home_candidate = home_candidate.cluster.values[0]
            #     #print(home_candidate.cluster.values)
            #     other_candidates = other_candidates[~other_candidates.cluster.isin(home_candidate)]
            #     data.iloc[i+1, 1] = home_candidate




                # other_candidates = other_candidates[other_candidates['cluster'].]
                # data.iloc[i+1, 'cluster']= home_candidates


    return data


tqdm.pandas()
df_survey_act_modified = ppl_other_new.groupby('uid').progress_apply(seq_modifier)

  0%|          | 0/114315 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
[1,2,3,4,5]

IndexError: list index out of range

In [93]:
# df_act_schedule = pd.DataFrame(combined_df.act_seq_simp_0.tolist(), index=combined_df['uid'])
#
# df_act_schedule = df_act_schedule.stack()
#
# df_act_schedule = pd.DataFrame(df_act_schedule)
#
# df_act_schedule = df_act_schedule.reset_index()
#
# df_act_schedule = df_act_schedule.rename(columns={'level_0': 'PId', 'level_1': 'act_id', 0: 'act_purpose'})